### Package Imports

In [173]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from pyproj import Transformer, Proj, transform
import pyproj
from tqdm.notebook import tqdm
from shapely.geometry import Point as point
import geopandas as gpd
import folium

In [175]:
df1 = pd.read_csv(f'C:\\Users\\Administrator\\Desktop\\Projects\\raw_data\\일반음식점.csv', engine='python', encoding = 'CP949', sep=',', quotechar='"', error_bad_lines=False)
df2 = pd.read_csv(f'C:\\Users\\Administrator\\Desktop\\Projects\\raw_data\\제과점영업.csv', engine='python', encoding = 'CP949', sep=',', quotechar='"', error_bad_lines=False)
df3 = pd.read_csv(f'C:\\Users\\Administrator\\Desktop\\Projects\\raw_data\\휴게음식점.csv', engine='python', encoding = 'CP949', sep=',', quotechar='"', error_bad_lines=False)

Skipping line 79337: ',' expected after '"'
Skipping line 147128: ',' expected after '"'
Skipping line 148855: ',' expected after '"'
Skipping line 148975: ',' expected after '"'
Skipping line 179473: ',' expected after '"'
Skipping line 298875: ',' expected after '"'
Skipping line 310404: ',' expected after '"'
Skipping line 327946: ',' expected after '"'
Skipping line 334357: ',' expected after '"'
Skipping line 368065: ',' expected after '"'
Skipping line 431143: ',' expected after '"'
Skipping line 477015: ',' expected after '"'
Skipping line 601552: ',' expected after '"'
Skipping line 644246: ',' expected after '"'
Skipping line 646495: ',' expected after '"'
Skipping line 738422: ',' expected after '"'
Skipping line 781724: ',' expected after '"'
Skipping line 800983: ',' expected after '"'
Skipping line 830798: ',' expected after '"'
Skipping line 836841: ',' expected after '"'
Skipping line 901266: ',' expected after '"'
Skipping line 901898: ',' expected after '"'
Skipping li

### 3개의 csv 파일의 데이터를 df로 모음.

In [130]:
df = pd.concat([df1, df2, df3])
df.head()

,번호,개방서비스명,개방서비스아이디,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,...,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지,Unnamed: 47
0,1,일반음식점,07_24_04_P,4200000,4200000-101-2022-00123,20220523,NaN,3,폐업,2,...,0.0,NaN,0.0,0.0,N,0.0,NaN,NaN,NaN,NaN
1,2,일반음식점,07_24_04_P,3010000,3010000-101-2022-00135,20220427,NaN,3,폐업,2,...,0.0,NaN,0.0,0.0,N,0.0,NaN,NaN,NaN,NaN
2,3,일반음식점,07_24_04_P,3450000,3450000-101-2022-00127,20220428,NaN,3,폐업,2,...,0.0,NaN,0.0,0.0,N,0.0,NaN,NaN,NaN,NaN
3,4,일반음식점,07_24_04_P,3700000,3700000-101-2022-00133,20220428,NaN,3,폐업,2,...,0.0,NaN,0.0,0.0,N,0.0,NaN,NaN,NaN,NaN
4,5,일반음식점,07_24_04_P,3740000,5610000-101-2022-00139,20220428,NaN,3,폐업,2,...,0.0,NaN,0.0,0.0,N,0.0,NaN,NaN,NaN,NaN


### DATA 수

In [131]:
len(df)

2554725

### Columns 확인

In [132]:
df.columns

Index(['번호', '개방서비스명', '개방서비스아이디', '개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자',
       '영업상태구분코드', '영업상태명', '상세영업상태코드', '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자',
       '재개업일자', '소재지전화', '소재지면적', '소재지우편번호', '소재지전체주소', '도로명전체주소', '도로명우편번호',
       '사업장명', '최종수정시점', '데이터갱신구분', '데이터갱신일자', '업태구분명', '좌표정보(x)', '좌표정보(y)',
       '위생업태명', '남성종사자수', '여성종사자수', '영업장주변구분명', '등급구분명', '급수시설구분명', '총종업원수',
       '본사종업원수', '공장사무직종업원수', '공장판매직종업원수', '공장생산직종업원수', '건물소유구분명', '보증액',
       '월세액', '다중이용업소여부', '시설총규모', '전통업소지정번호', '전통업소주된음식', '홈페이지',
       'Unnamed: 47'],
      dtype='object')

### 필요없는 Columns Drop

In [133]:
dropping_list = ['번호', '관리번호','인허가일자','인허가취소일자','보증액','월세액','홈페이지','전통업소주된음식','시설총규모',
                 '휴업시작일자', '휴업종료일자','최종수정시점','데이터갱신일자','위생업태명','남성종사자수','여성종사자수','영업장주변구분명',
                 'Unnamed: 47','본사종업원수','공장사무직종업원수','공장판매직종업원수','공장생산직종업원수','건물소유구분명',
                 '다중이용업소여부','재개업일자','소재지전화','도로명우편번호','데이터갱신구분','등급구분명','총종업원수',
                 '전통업소지정번호','급수시설구분명','소재지면적','상세영업상태명','영업상태구분코드','상세영업상태코드',
                 '개방서비스아이디','개방자치단체코드','폐업일자','소재지우편번호'
                ]

In [134]:
df = df.drop(columns = dropping_list)

In [135]:
df.head()

,개방서비스명,영업상태명,소재지전체주소,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y)
0,일반음식점,폐업,강원도 강릉시 노암동 825-2,NaN,2022 강릉단오제 먹거리존,한식,366964.031622,473505.518622
1,일반음식점,폐업,서울특별시 중구 을지로6가 17-2,"서울특별시 중구 장충단로13길 20, 지하2층 (을지로6가)",금덕푸드,기타,200613.510298,451817.515367
2,일반음식점,폐업,대구광역시 북구 산격동 1676 대구전시컨벤션센터,"대구광역시 북구 엑스코로 10, 대구전시컨벤션센터 (산격동)",오사카푸드,일식,345549.110170,268620.845678
3,일반음식점,폐업,울산광역시 남구 삼산동 1521-1 현대백화점,"울산광역시 남구 삼산로 261, 현대백화점 지하1층 (삼산동)",프띠르,한식,411768.040749,229164.489593
4,일반음식점,폐업,경기도 수원시 영통구 하동 1017-2 갤러리아 광교,"경기도 수원시 영통구 광교중앙로 124, 갤러리아 광교 지1층 (하동)",편장군족발 갤리러아광교,기타,NaN,NaN


### 폐업한 곳의 정보는 필요 없으니 Drop

In [136]:
df = df[df['영업상태명']!='폐업']
df = df.reset_index(drop=True)

### 필요없는 column 추가 제거하고 데이터를 보기 편하도록 순서변경

In [137]:
df = df.drop(columns = ['영업상태명','개방서비스명'])
df = df[['사업장명', '소재지전체주소', '도로명전체주소', '좌표정보(x)', '좌표정보(y)','업태구분명']]

In [138]:
df

,사업장명,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),업태구분명
0,윤푸드,경기도 성남시 수정구 신흥동 2562-2 1층,"경기도 성남시 수정구 산성대로295번길 9, 1(일부)층 (신흥동)",213117.592076,437825.856238,한식
1,79대포 안성공도점,경기도 안성시 공도읍 만정리 815-1 107호,"경기도 안성시 공도읍 공도3로 9-9, 공도 일번가타워 1층 107호",215021.103199,388727.667964,호프/통닭
2,맛있어2(Masisso2),경기도 안성시 대덕면 내리 691-6 2층 일부,"경기도 안성시 대덕면 중앙대학로 116-1, 2층",219857.937502,388861.268106,한식
3,푸드 딜리버리 영등포점,서울특별시 영등포구 도림동 76-22,"서울특별시 영등포구 도신로33길 11, 1층 (도림동)",191722.709234,445276.999732,한식
4,수벙샌드위치,경상남도 양산시 물금읍 물금리 878-6,"경상남도 양산시 물금읍 버들2길 7-13, 1층 일부",380741.211631,202644.423181,기타
...,...,...,...,...,...,...
905012,GS25HR아팰리스점,광주광역시 남구 봉선동 1095 1층 201동 102호,"광주광역시 남구 제석로 104, 201동 1층 102호 (봉선동)",191473.699104,180506.254787,편의점
905013,석류나무,광주광역시 남구 백운동 620-11번지 1층,"광주광역시 남구 독립로 30-3, 1층 (백운동)",190987.643081,181915.397881,일반조리판매
905014,블랑PC방,"광주광역시 남구 진월동 294-17번지 광동빌딩, 2층","광주광역시 남구 서문대로 700, 광동빌딩 2층 (진월동)",190720.873936,180300.549186,일반조리판매
905015,엔젤pc,광주광역시 남구 월산동 901-30번지 1층,"광주광역시 남구 월산로151번길 10, 땡큐pc방 1층 (월산동)",190337.038953,183555.773219,기타 휴게음식점


### 결측치 제거

In [147]:
print(df['사업장명'].isna().sum())
df = df.dropna()

0


업태 중 **까페, 다방, 전통찻집,커피숍** 만을 선택

In [148]:
set(df['업태구분명'])

{'감성주점',
 '경양식',
 '고속도로',
 '공항',
 '과자점',
 '관광호텔',
 '극장',
 '기타',
 '기타 휴게음식점',
 '김밥(도시락)',
 '까페',
 '냉면집',
 '다방',
 '떡카페',
 '라이브카페',
 '백화점',
 '복어취급',
 '분식',
 '뷔페식',
 '식육(숯불구이)',
 '아이스크림',
 '외국음식전문점(인도,태국등)',
 '유원지',
 '이동조리',
 '일반조리판매',
 '일식',
 '전통찻집',
 '정종/대포집/소주방',
 '제과점영업',
 '중국식',
 '철도역구내',
 '출장조리',
 '커피숍',
 '키즈카페',
 '탕류(보신용)',
 '통닭(치킨)',
 '패밀리레스트랑',
 '패스트푸드',
 '편의점',
 '푸드트럭',
 '한식',
 '호프/통닭',
 '횟집'}

In [149]:
coffee_df = pd.concat([
    df[df['업태구분명']=='까페'],
    df[df['업태구분명']=='다방'],
    df[df['업태구분명']=='전통찻집'],
    df[df['업태구분명']=='커피숍']]
    )
coffee_df=coffee_df.reset_index(drop=True)
coffee_df

,사업장명,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),업태구분명
0,미엔,대구광역시 달서구 상인동 1419-6,"대구광역시 달서구 월곡로45길 46, 1층 (상인동)",339048.796303,258157.087819,까페
1,(주)맨케이브,서울특별시 마포구 공덕동 111-127,"서울특별시 마포구 임정로 81, 2층 (공덕동)",196379.695803,449331.505387,까페
2,청자,전라남도 곡성군 곡성읍 읍내리 270-1,전라남도 곡성군 곡성읍 중앙로 110,226742.667209,197980.499485,까페
3,레이지민트,"경기도 성남시 분당구 서현동 260-4 덕산빌딩 2층 201,203호","경기도 성남시 분당구 황새울로335번길 8, 덕산빌딩 2층 201,203호 (서현동)",210745.282176,431619.243033,까페
4,딤플,서울특별시 성동구 성수동2가 228-3,"서울특별시 성동구 성덕정길 143, 1층 (성수동2가)",205174.619649,448266.234572,까페
...,...,...,...,...,...,...
91211,스타벅스동대구터미널점,"대구광역시 동구 신천동 330-6 번지외1필지 보광빌딩 가동 1,2층","대구광역시 동구 동부로30길 12 (신천동,번지외1필지 보광빌딩 가동 1,2층)",347004.198893,265224.513184,커피숍
91212,메가엠지씨커피 광주봉선유안공원점,광주광역시 남구 봉선동 138-2 (1층)103호,"광주광역시 남구 봉선로 205 (봉선동,(1층)103호)",192427.177049,180427.408679,커피숍
91213,데이하우스,광주광역시 서구 화정동 824-17번지 (1층),"광주광역시 서구 월드컵4강로 70-1 (화정동, 1층)",188491.350802,182697.968349,커피숍
91214,벌크커피봉선점,광주광역시 남구 봉선동 606번지 남양휴턴1차 106동 107호,"광주광역시 남구 봉선로 106, 106동 107호 (봉선동, 남양휴턴1차 지하1)",191414.807157,180524.245780,커피숍


### 현재 제공되는 좌표 : epsg:2097. 추후 효과적인 데이터 분석을 위해 epsg:4326로 변환한다

In [161]:


def Loc_format_chg(df):
    
    # Projection 정의
    # 중부원점(Bessel): 서울 등 중부지역 EPSG:2097
    proj_1 = Proj(init='epsg:2097')

    # WGS84 경위도: GPS가 사용하는 좌표계 EPSG:4326
    proj_2 = Proj(init='epsg:4326')

    DataFrame = df.copy()

    x_list = []
    y_list = []

    for idx, row in tqdm(DataFrame.iterrows()):
        x, y = row['좌표정보(x)'], row['좌표정보(y)']
        x_, y_ = transform(proj_1, proj_2, x, y)
        x_list.append(x_)
        y_list.append(y_)

    df['lon'] = x_list
    df['lat'] = y_list
    
    return df

In [163]:
coffee_df = Loc_format_chg(coffee_df)
coffee_df.head()

0it [00:00, ?it/s]

,사업장명,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),업태구분명,lon,lat
0,미엔,대구광역시 달서구 상인동 1419-6,"대구광역시 달서구 월곡로45길 46, 1층 (상인동)",339048.796303,258157.087819,까페,128.536416,35.813690
1,(주)맨케이브,서울특별시 마포구 공덕동 111-127,"서울특별시 마포구 임정로 81, 2층 (공덕동)",196379.695803,449331.505387,까페,126.956931,37.546238
2,청자,전라남도 곡성군 곡성읍 읍내리 270-1,전라남도 곡성군 곡성읍 중앙로 110,226742.667209,197980.499485,까페,127.291871,35.280803
3,레이지민트,"경기도 성남시 분당구 서현동 260-4 덕산빌딩 2층 201,203호","경기도 성남시 분당구 황새울로335번길 8, 덕산빌딩 2층 201,203호 (서현동)",210745.282176,431619.243033,까페,127.119233,37.386595
4,딤플,서울특별시 성동구 성수동2가 228-3,"서울특별시 성동구 성덕정길 143, 1층 (성수동2가)",205174.619649,448266.234572,까페,127.056446,37.536633


In [164]:
coffee_df=coffee_df.drop(columns=['좌표정보(x)','좌표정보(y)'])
coffee_df

,사업장명,소재지전체주소,도로명전체주소,업태구분명,lon,lat
0,미엔,대구광역시 달서구 상인동 1419-6,"대구광역시 달서구 월곡로45길 46, 1층 (상인동)",까페,128.536416,35.813690
1,(주)맨케이브,서울특별시 마포구 공덕동 111-127,"서울특별시 마포구 임정로 81, 2층 (공덕동)",까페,126.956931,37.546238
2,청자,전라남도 곡성군 곡성읍 읍내리 270-1,전라남도 곡성군 곡성읍 중앙로 110,까페,127.291871,35.280803
3,레이지민트,"경기도 성남시 분당구 서현동 260-4 덕산빌딩 2층 201,203호","경기도 성남시 분당구 황새울로335번길 8, 덕산빌딩 2층 201,203호 (서현동)",까페,127.119233,37.386595
4,딤플,서울특별시 성동구 성수동2가 228-3,"서울특별시 성동구 성덕정길 143, 1층 (성수동2가)",까페,127.056446,37.536633
...,...,...,...,...,...,...
91211,스타벅스동대구터미널점,"대구광역시 동구 신천동 330-6 번지외1필지 보광빌딩 가동 1,2층","대구광역시 동구 동부로30길 12 (신천동,번지외1필지 보광빌딩 가동 1,2층)",커피숍,128.625708,35.876201
91212,메가엠지씨커피 광주봉선유안공원점,광주광역시 남구 봉선동 138-2 (1층)103호,"광주광역시 남구 봉선로 205 (봉선동,(1층)103호)",커피숍,126.914840,35.122915
91213,데이하우스,광주광역시 서구 화정동 824-17번지 (1층),"광주광역시 서구 월드컵4강로 70-1 (화정동, 1층)",커피숍,126.871630,35.143344
91214,벌크커피봉선점,광주광역시 남구 봉선동 606번지 남양휴턴1차 106동 107호,"광주광역시 남구 봉선로 106, 106동 107호 (봉선동, 남양휴턴1차 지하1)",커피숍,126.903733,35.123780


In [ ]:
points = gpd.GeoDataFrame(geometry = (coffee_df['lon'],coffee_df['lat']))
points.to_file('C:/Users/user/Desktop/tta_points.shp', driver="Shapefile")